In [ ]:
# add root folder of the project to path
import sys
sys.path.insert(0, '../..')

# Data loading

In [ ]:
# parameter settings
is_plot = True
is_export = True

landmarks_path = '../../data/landmarks/refine_6kmh_braless_18markers_12fps.pkl'
meshes_path = '../../data/meshes/6kmh_braless_26markers/'
test_landmarks_path = '../../data/test/braless_random_landmarks.pkl'

start=0
stride = 12
end=120

In [ ]:
from mesh4d import obj3d

mesh_ls, texture_ls = obj3d.load_mesh_series(
    folder=meshes_path,
    start=start,
    stride=stride,
    end=end,
)

In [ ]:
from mesh4d import utils

landmarks = utils.load_pkl_object(landmarks_path)
landmarks.interp_field()

In [ ]:
from mesh4d.analyse.crave import clip_with_contour

contour = landmarks.extract(('marker 0', 'marker 2', 'marker 15', 'marker 17'))
mesh_clip_ls = clip_with_contour(mesh_ls, start_time=0, fps=12, contour=contour, clip_bound='xy', margin=0.5)

In [ ]:
body_ls = obj3d.init_obj_series(
    mesh_ls, 
    obj_type=obj3d.Obj3d_Deform
)

In [ ]:
breast_ls = obj3d.init_obj_series(
    mesh_clip_ls, 
    obj_type=obj3d.Obj3d_Deform
    )

# Sampling point clouds

In [ ]:
pcd0 = breast_ls[0].get_vertices()
pcd1 = breast_ls[1].get_vertices()

In [ ]:
pcd0 = breast_ls[0].get_sample_points(1000)
pcd1 = breast_ls[1].get_sample_points(1000)

# `probreg` registration

## Coherent Point Drift

In [ ]:
from probreg import cpd
tf_param, _, _ = cpd.registration_cpd(pcd0, pcd1)
pcd_tf = tf_param.transform(pcd0)

## Bayesian Coherent Point Drift

In [ ]:
from probreg import bcpd
tf_param = bcpd.registration_bcpd(pcd0, pcd1)
pcd_tf = tf_param.transform(pcd0)

In [ ]:
import pyvista as pv
scene = pv.Plotter()
# scene.add_points(pcd0, point_size=1e-3, color='gold')
scene.add_points(pcd1, point_size=1e-3, color='green')
scene.add_points(pcd_tf, point_size=1e-3, color='blue')
scene.show()